In [ ]:
# nst_demo.ipynb

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image as kp_image
from tensorflow.keras.applications import vgg19
import numpy as np
import matplotlib.pyplot as plt
import time
import os



# ========== CONFIG ==========

content_path = "content.jpg"
style_path = "monet.jpg"
result_prefix = "outputs/monet_city"
iterations = 350  
opt = tf.optimizers.Adam(learning_rate=0.02)

# key hyper‑parameters
style_weight        = 1e-2   # β
content_weight      = 1e4    # α
total_variation_weight = 30  # γ

# ========== IMAGE UTILS ==========

def load_img(path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim
    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

def imshow(tensor, title=None):
    img = tensor.numpy()
    img = img.squeeze()
    plt.imshow(img)
    if title:
        plt.title(title)
    plt.axis('off')
    plt.show()

def tensor_to_image(tensor):
    tensor = tensor * 255
    tensor = tf.clip_by_value(tensor, 0, 255)
    tensor = tf.cast(tensor, tf.uint8)
    return tf.image.encode_png(tf.squeeze(tensor))

# ========== LOAD VGG MODEL ==========

def vgg_layers(layer_names):
    """Creates a VGG model that returns a list of intermediate output values."""
    vgg = vgg19.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    outputs = [vgg.get_layer(name).output for name in layer_names]
    model = tf.keras.Model([vgg.input], outputs)
    return model

# ========== EXTRACT FEATURES ==========

content_layers = ['block5_conv2']
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
num_content = len(content_layers)
num_style = len(style_layers)

def gram_matrix(input_tensor):
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result / num_locations

class StyleContentModel(tf.keras.models.Model):
    def __init__(self, style_layers, content_layers):
        super().__init__()
        self.vgg = vgg_layers(style_layers + content_layers)
        self.style_layers = style_layers
        self.content_layers = content_layers
        self.vgg.trainable = False

    def call(self, inputs):
        inputs = inputs * 255.0
        preprocessed = vgg19.preprocess_input(inputs)
        outputs = self.vgg(preprocessed)
        style_outputs, content_outputs = (outputs[:num_style], outputs[num_style:])
        style_outputs = [gram_matrix(style_output) for style_output in style_outputs]

        content_dict = {content_name:value 
                        for content_name, value in zip(self.content_layers, content_outputs)}

        style_dict = {style_name:value
                      for style_name, value in zip(self.style_layers, style_outputs)}

        return {'content': content_dict, 'style': style_dict}

# ========== STYLE TRANSFER ==========

def clip_0_1(image):
    return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

def style_content_loss(outputs):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                           for name in style_outputs.keys()])
    style_loss *= style_weight / num_style

    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                             for name in content_outputs.keys()])
    content_loss *= content_weight / num_content
    loss = style_loss + content_loss
    return loss

@tf.function()
def train_step(image):
    with tf.GradientTape() as tape:
        outputs = extractor(image)
        loss = style_content_loss(outputs)
        loss += total_variation_weight * tf.image.total_variation(image)
    grad = tape.gradient(loss, image)
    opt.apply_gradients([(grad, image)])
    image.assign(clip_0_1(image))
    return loss

# ========== LOAD IMAGES ==========

content_image = load_img(content_path)
style_image = load_img(style_path)

extractor = StyleContentModel(style_layers, content_layers)
style_targets = extractor(style_image)['style']
content_targets = extractor(content_image)['content']

image = tf.Variable(content_image)



start = time.time()

step = 0
for n in range(iterations):
    step += 1
    loss = train_step(image)
    if (n + 1) % 50 == 0 or (n + 1) == 1:
        img = tensor_to_image(image)
        output_file = f"{result_prefix}_at_iteration_{n+1}.png"
        tf.io.write_file(output_file, img)
        print(f"iter {n+1}: loss={float(loss.numpy()):.2e} → saved {output_file}")


end = time.time()
print("Total time: {:.1f} seconds".format(end - start))


C:\Users\kimch\AppData\Local\Temp\ipykernel_96424\3324553136.py:155: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"iter {n+1}: loss={float(loss.numpy()):.2e} → saved {output_file}")


iter 1: loss=1.19e+08 → saved outputs/monet_city_at_iteration_1.png
iter 50: loss=5.45e+06 → saved outputs/monet_city_at_iteration_50.png
iter 100: loss=3.80e+06 → saved outputs/monet_city_at_iteration_100.png
iter 150: loss=3.32e+06 → saved outputs/monet_city_at_iteration_150.png
iter 200: loss=3.05e+06 → saved outputs/monet_city_at_iteration_200.png
iter 250: loss=2.89e+06 → saved outputs/monet_city_at_iteration_250.png
iter 300: loss=2.98e+06 → saved outputs/monet_city_at_iteration_300.png
iter 350: loss=2.87e+06 → saved outputs/monet_city_at_iteration_350.png
Total time: 707.0 seconds


In [3]:
# --- SSIM evaluation ---
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_for_ssim(path):
    img = load_img(path, target_size=(image.shape[1], image.shape[2]))
    img = img_to_array(img) / 255.0
    return img

content_for_ssim = load_for_ssim(content_path)
result_for_ssim = load_for_ssim(f"{result_prefix}_at_iteration_{iterations}.png")

score = ssim(content_for_ssim, result_for_ssim, channel_axis=2, data_range=1.0)
print(f"SSIM (content vs final output): {score:.4f}")

SSIM (content vs final output): 0.4882
